In [ ]:
!pip install gymnasium[classic-control]

In [ ]:
import numpy as np
import gymnasium as gym
import tensorflow as tf
from tensorflow import keras
from collections import deque

In [ ]:
GAMMA = 0.99
LEARNING_RATE = 0.001
MEMORY_SIZE = 1000000
BATCH_SIZE = 64
EXPLORATION_MAX = 1.0
EXPLORATION_MIN = 0.01
EXPLORATION_DECAY = 0.995

In [ ]:
class DQNAgent:
  def __init__(self, observation_space, action_space):
    self.exploration_rate = EXPLORATION_MAX
    self.action_space = action_space

    self.memory = deque(maxlen=MEMORY_SIZE)

    self.model = keras.Sequential()
    self.model.add(keras.layers.Dense(24, input_shape=(observation_space,), activation="relu"))
    self.model.add(keras.layers.Dense(24, activation="relu"))
    self.model.add(keras.layers.Dense(self.action_space, activation="linear"))

    self.model.compile(loss="mse", optimizer=keras.optimizers.Adam(learning_rate=LEARNING_RATE))

  def remember(self, state, action, reward, next_state, done):
    self.memory.append((state, action, reward, next_state, done))

  def act(self, state):
    if np.random.rand() < self.exploration_rate:
      return np.random.randint(self.action_space)
    q_values = self.model.predict(state)
    return np.argmax(q_values[0])

  def experience_replay(self):
    if len(self.memory) < BATCH_SIZE:
      return

    batch = np.random.choice(len(self.memory), BATCH_SIZE, replace = False)

    for i in batch:
      state, action, reward, next_state, done = self.memory[i]
      q_update = reward

      if not done:
        q_update = (reward + GAMMA * np.amax(self.model.predict(next_state)[0]))

      q_values = self.model.predict(state)
      q_values[0][action] = q_update
      self.model.fit(state, q_values, verbose=0)

    self.exploration_rate *= EXPLORATION_DECAY
    self.exploration_rate = max(EXPLORATION_MIN, self.exploration_rate)